In [1]:
from mindquantum.core.circuit import Circuit, UN, add_suffix, apply, decompose_single_term_time_evolution
from mindquantum.core.gates import H, Rzz, RX, Rzz,RZ, RY, Measure, BarrierGate
from mindquantum.core.operators import Hamiltonian, QubitOperator
from mindquantum.framework import MQAnsatzOnlyLayer
from mindquantum.simulator import Simulator
from mindquantum.core.operators import commutator
import time
from mindquantum.core.parameterresolver import ParameterResolver
from mindquantum import *

import mindspore.nn as nn
from mindspore import Tensor
import mindspore as ms
ms.set_context(mode=ms.PYNATIVE_MODE, device_target="CPU")

import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import matplotlib.pylab as pylab
from tqdm import tqdm

from itertools import combinations
from collections import OrderedDict as ordict

## Build quantum circuits

In [2]:
def param2dict(keys, values):
    # make the parameters and coresponding label to a dict
    param_dict = {}
    for (key, value) in zip(keys, values):
        param_dict[key] = value
    return param_dict

def get_expectation_of_hamitonian(circ, ham, pr):
    # calculate the expectation of hamitonian
    sim = Simulator('mqvector', circ.n_qubits)
    sim.apply_circuit(circ, pr)
    result = sim.get_expectation(ham)
        
    return result.real
def E0_energy(ham_operator):
    
    # 首先根据哈密顿量的算符将其转化为矩阵形式。
    ham_matrix = ham_operator.matrix()
    ham_matrix = ham_matrix.todense()

    # calculate the eigenvalue and eigenvector
    eigval, eigvec = np.linalg.eig(ham_matrix)

    # ground energy
    E0 = min(eigval).real
    #print(eigval, eigvec)
    
    return E0

def train(ham, ansatze, iteration):

    # bulid the quantum circuit
    
    circ = ansatze                                         # 将初始化线路与ansatz线路组合成一个线路
    sim = Simulator('mqvector', ansatze.n_qubits)  
    #sim.set_qs(initial_ground_state)
    
    grad_ops = sim.get_expectation_with_grad(ham, circ)    # 获取计算变分量子线路的期望值和梯度的算子
    net = MQAnsatzOnlyLayer(grad_ops)                             
    opti = nn.Adam(net.trainable_params(), learning_rate=0.05)     
    train_net = nn.TrainOneStepCell(net, opti)                    

    result = []
    #from tqdm import tqdm
    for i in range(iteration):
        train_net() 
        result.append(np.array(train_net()[0]))
    pr = dict(zip(ansatze.params_name, net.weight.asnumpy())) # 获取线路参数
        
    return pr, result


def state_sampling(ansatze, pr):
    
    shots = 1024

    for i in range(ansatze.n_qubits):
        ansatze += Measure().on(i)
    

    sim = Simulator('mqvector', ansatze.n_qubits)
    #sim.set_qs(initial_ground_state)
    res = sim.sampling(ansatze, pr, shots=shots)
    
    return res

def CD_operator(CD):
    
    CD_term = []
    for cd in CD.split():
        CD_term.append(cd[1])
    k = 0
    out = ordict()
    for cd in CD_term:
        for key, term in cd.terms.items():
            if len(key) == 2:
                #print(len(key))
                if key not in out:
                    out[key] = ordict({f"p{k}": 1})
                    k += 1
    return out

def CD_ansataz(out):
    "Transform a pauli ansatz to parameterized quantum circuit."

    circuit = Circuit()
    # circuit += inital_circuits(occupied, circuit_description)
        
    z = 0
    for k, v in out.items():
        index1 = (k[0][0], k[1][0])
        index2 = (k[1][0], k[0][0])

        pauli_operator = (str(k[0][1]), str(k[1][1]))
        if pauli_operator == ('Z','Y'):
            circuit += Ryz(f'{z}').on(index2)          
            z+=1
    # for k, v in out.items():
    #     circuit += decompose_single_term_time_evolution(k, v)
    for i in range(n_items):
        circuit += RY(f'ry{i}').on(i)
    
    return circuit

### The Hamitonian 
$H_p = \sum_{i=1}^{n}(-\frac{\alpha}{2}+\beta(C-\frac{1}{2}\sum_{i}^{n}\omega_{i}))Z_{i}\omega_{i} + \sum_{i<j}\frac{\beta}{2}\omega_{i}\omega_{j}Z_{i}Z_{j}.$

In [3]:
def generate_h_one_body(n, W, alpha, beta, capacity):
    ele1 = QubitOperator()
    for i in range(n):
            ele1 += QubitOperator(f'Z{i}',-W[i]*(alpha/2 - beta*capacity))
    for i in range(n):
        for j in range(n):
            ele1 += QubitOperator(f'Z{i}',-1/2*beta*W[i]*W[j])
    return ele1

def generate_h_two_body(n, W, beta):
    ele1 = QubitOperator()
    h_two_body = QubitOperator()
    for i in range(n):
        for j in range(n):
            if i < j:
                ele1 += QubitOperator(f'Z{i} Z{j}',W[i]*W[j]*beta/2)
    return ele1

def generate_h_mixer(n):
    h_mixer = QubitOperator()
    for i in range(n):
        h_mixer += QubitOperator(f'X{i}')
    return h_mixer

In [4]:
# Function of generating all possible solution in one bin
def find_combinations(elements, target_sum):
    result = []
    # Enumerate the elements to keep track of their indices
    enumerated_elements = list(enumerate(elements))
    for r in range(1, len(elements) + 1):
        for combo in combinations(enumerated_elements, r):
            # Calculate the sum of the current combination
            combo_sum = sum(ele[1] for ele in combo)
            if combo_sum <= target_sum:
                # Store the indices of the elements in the combination
                result.append([ele[0] for ele in combo])
    return result


# Function of calculating the minimum positive difference of weights
def min_positive_difference(lst):
    # Sort the list in ascending order
    sorted_lst = sorted(lst)
    
    # Initialize min_diff to a large value
    min_diff = float('inf')

    # Iterate through the sorted list to find the minimum positive difference
    for i in range(len(sorted_lst) - 1):
        diff = sorted_lst[i + 1] - sorted_lst[i]
        if 0 < diff < min_diff:
            min_diff = diff

    return min_diff if min_diff != float('inf') else None

### check the minimum number of bin

In [5]:
def find_min_number(lists, target):
    opti_solu = 0
    for n in tqdm(range(n_items)):
        nested_list = list(combinations(lists,n))
        for sublist in nested_list:
            l = []
            for i in sublist:
                l+=i
                l = sorted(l)
                if l==target:
                    opti_solu += 1
        if opti_solu != 0:
            return n, opti_solu
            break
    return 999, 999

### define the traing process, including optimizer, iteration

In [6]:
def Quantum_cd_mixer(h_m, ci, n_layer, n_items):
       
    u_m = 0
    index=0
    for i in h_m:
        u_m += decompose_single_term_time_evolution(i, ParameterResolver(f'c{index}'))
        index=index+1

    p = n_layer
    prep_circ = UN(H, n_items)
    ansatz_template =   u_m + ci #u_p +
    ansatz = Circuit() + prep_circ

    for i in range(p):
        ansatz += add_suffix(ansatz_template, str(i)) + BarrierGate()
        
    return ansatz

def Quantum_QAOA_Circuits(h_m, h_p, n_layer, n_items):
    
            
    u_m = 0
    index=0
    for i in h_m:
        u_m += decompose_single_term_time_evolution(i, ParameterResolver(f'b{index}'))
        index=index+1

    u_p = 0
    index=0
    for i in h_p:
        u_p += decompose_single_term_time_evolution(i, ParameterResolver(f'a{index}'))
        index=index+1


    p = n_layer
    prep_circ = UN(H, n_items)
    ansatz_template =   u_p + u_m 
    ansatz = Circuit() + prep_circ

    for i in range(p):
        ansatz += add_suffix(ansatz_template, str(i)) + BarrierGate()
        
    return ansatz

In [7]:
def data_analyze(n_items, solution_set, W, Capa, target_solu):
    
    # Extracting keys with values greater than 100
    data = [key for sub_dict in solution_set for key, value in sub_dict.items()]
    # Remove the duplicate keys
    result_list = list(set(data))
    # Calculate the weight sum for each binary string
    weight_sums = [sum(W[i] for i in range(len(result_list_sub)) if result_list_sub[i] == '1') for result_list_sub in result_list]
    # Combine the binary strings and their corresponding weight sums
    combined_data = list(zip(result_list, weight_sums))
    #print(combined_data)
    # Filter out the empty lists
    filtered_empty = [(binary_string, value) for binary_string, value in combined_data if value != 0]
    #print(filtered_empty)
    # Filter out the data whose weight sum is bigger than Capacity
    filtered_data = [(binary_string, weight_sum) for binary_string, weight_sum in filtered_empty if weight_sum <= Capa]
    # Sort the combined data by weight sum in descending order
    sorted_data = sorted(filtered_data, key=lambda x: x[1], reverse=True)
    # Read the feasible solutions
    sorted_binary_strings = [data[0] for data in sorted_data] 
    
    ar = len(sorted_binary_strings)/len(target_solu)
    sq = len(sorted_binary_strings)/len(filtered_empty)

    return  sorted_binary_strings, ar, sq

In [8]:
#dk = 1.5
ite = 100
n_layer = 1

dk_list = [2.5]
#iteration_list = [100, 150, 200]
#n_layer_list = [1, 2, 3]

In [9]:
n_items = 10 #number of items
capacity = 120 #maximum capacity of box

# insert QAOA weight list to compare fairly
W = [42, 44, 63, 41, 51, 45, 42, 69, 50, 40]

#np.random.normal(loc=capacity/2, size=n)
beta = np.min(W)/5
delta_w = min_positive_difference(W)

# generate exact solutions
result_combinations = find_combinations(W, capacity)
target_solu = list(result_combinations)
target = list(range(n_items))

In [10]:
batch = 5
sr_set = np.zeros((len(dk_list), batch))
sq_set = np.zeros((len(dk_list), batch))
n_solu_set = np.zeros((len(dk_list), batch))
minbin_set = np.zeros((len(dk_list), batch))


for i_index, dk in enumerate(dk_list):

    for j in range(batch):

        solution_set = []
        # qu_time = []
        # start_time_process1 = time.time()

        for k in tqdm(np.arange(1,(capacity/delta_w),dk)):
            alpha = 2*beta*(capacity-k*delta_w)
            h_p = generate_h_two_body(n_items, W, beta) + generate_h_one_body(n_items, W, alpha, beta, capacity)
            h_m = generate_h_mixer(n_items)
            h_cd = commutator(h_m,h_p)#CD pool
            cd = CD_operator(h_cd)
            ci = CD_ansataz(cd)

            circ = Quantum_QAOA_Circuits(h_m, h_p, n_layer, n_items)
            #ground_energy.append(E0_energy(h_p))

            ham = Hamiltonian(h_p)
            sim = Simulator('mqvector', n_items)
            grad_ops = sim.get_expectation_with_grad(ham, circ)

            pr, result = train(ham, circ, ite)
            #cost_fun.append(result)
            res = state_sampling(circ, pr)
            solution_set.append(res.data)

        # Record the current time after the first process
        # end_time_process1 = time.time()

        # Calculate the elapsed time for the quantum process
        # elapsed_time_process1 = end_time_process1 - start_time_process1
        # qu_time[i_index, j] = elapsed_time_process1

        elements_fea, succ_ratio, solution_quality= data_analyze(n_items, solution_set, W[::-1], capacity, target_solu)
        positions = [[n_items-1-i for i, char in enumerate(element) if char == '1'] for element in elements_fea]
        minbin, n_solu = find_min_number(positions,target)
        
        minbin_set[i_index, j] = minbin
        n_solu_set[i_index, j] = n_solu
        sr_set[i_index, j] = succ_ratio
        sq_set[i_index, j] = solution_quality

        # Record the current time before the second process
        # end_time_process2 = time.time()

        # Calculate the elapsed time for the classic process
        # elapsed_time_process2 = end_time_process2 - end_time_process1
        # cl_time[i_index, j] = elapsed_time_process2

        print('successful ratio is: ',succ_ratio)
        print('solution quality is: ',solution_quality)
        print('minimum number of bin is',minbin)
        # print("Elapsed time for quantum process:", np.mean(elapsed_time_process1), "seconds")
        # print("Elapsed time for classic process:", np.mean(elapsed_time_process2), "seconds")

 50%|█████     | 5/10 [00:01<00:01,  3.73it/s]


successful ratio is:  0.8148148148148148
solution quality is:  0.8301886792452831
minimum number of bin is 5


 50%|█████     | 5/10 [00:01<00:01,  4.23it/s]


successful ratio is:  0.7962962962962963
solution quality is:  0.7413793103448276
minimum number of bin is 5


 50%|█████     | 5/10 [00:01<00:01,  4.75it/s]


successful ratio is:  0.7777777777777778
solution quality is:  0.7368421052631579
minimum number of bin is 5


 50%|█████     | 5/10 [00:01<00:01,  3.47it/s]


successful ratio is:  0.8148148148148148
solution quality is:  0.7719298245614035
minimum number of bin is 5


 50%|█████     | 5/10 [00:01<00:01,  3.60it/s]

successful ratio is:  0.8148148148148148
solution quality is:  0.8301886792452831
minimum number of bin is 5


In [12]:
# Write data to a text file
with open('qaoa-k-W1.txt', 'w') as f:
    f.write("dk, batch, sr_set, sq_set, minbin_set, n_solu\n")
    for i_index, i in enumerate(dk_list):
        for j in range(batch):
            f.write(f"{i}, {j}, {sr_set[i_index, j]}, {sq_set[i_index, j]}, {minbin_set[i_index, j]}, {n_solu_set[i_index, j]}\n")

In [13]:
#dk = 1.5
ite = 100
n_layer = 1

dk_list = [0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4]
#iteration_list = [100, 150, 200]
#n_layer_list = [1, 2, 3]

In [34]:
n_items = 10 #number of items
capacity = 120 #maximum capacity of box

# insert QAOA weight list to compare fairly
W = [24, 35, 34, 23, 29, 20, 52, 33, 47, 20]

#np.random.normal(loc=capacity/2, size=n)
beta = np.min(W)/5
delta_w = min_positive_difference(W)

# generate exact solutions
result_combinations = find_combinations(W, capacity)
target_solu = list(result_combinations)
target = list(range(n_items))

In [35]:
len(target_solu)

246

In [15]:
batch = 5
sr_set = np.zeros((len(dk_list), batch))
sq_set = np.zeros((len(dk_list), batch))
n_solu_set = np.zeros((len(dk_list), batch))
minbin_set = np.zeros((len(dk_list), batch))


for i_index, dk in enumerate(dk_list):

    for j in range(batch):

        solution_set = []
        # qu_time = []
        # start_time_process1 = time.time()

        for k in tqdm(np.arange(1,(capacity/delta_w),dk)):
            alpha = 2*beta*(capacity-k*delta_w)
            h_p = generate_h_two_body(n_items, W, beta) + generate_h_one_body(n_items, W, alpha, beta, capacity)
            h_m = generate_h_mixer(n_items)
            h_cd = commutator(h_m,h_p)#CD pool
            cd = CD_operator(h_cd)
            ci = CD_ansataz(cd)

            circ = Quantum_QAOA_Circuits(h_m, h_p, n_layer, n_items)
            #ground_energy.append(E0_energy(h_p))

            ham = Hamiltonian(h_p)
            sim = Simulator('mqvector', n_items)
            grad_ops = sim.get_expectation_with_grad(ham, circ)

            pr, result = train(ham, circ, ite)
            #cost_fun.append(result)
            res = state_sampling(circ, pr)
            solution_set.append(res.data)

        # Record the current time after the first process
        # end_time_process1 = time.time()

        # Calculate the elapsed time for the quantum process
        # elapsed_time_process1 = end_time_process1 - start_time_process1
        # qu_time[i_index, j] = elapsed_time_process1

        elements_fea, succ_ratio, solution_quality= data_analyze(n_items, solution_set, W[::-1], capacity, target_solu)
        positions = [[n_items-1-i for i, char in enumerate(element) if char == '1'] for element in elements_fea]
        minbin, n_solu = find_min_number(positions,target)
        
        minbin_set[i_index, j] = minbin
        n_solu_set[i_index, j] = n_solu
        sr_set[i_index, j] = succ_ratio
        sq_set[i_index, j] = solution_quality

        # Record the current time before the second process
        # end_time_process2 = time.time()

        # Calculate the elapsed time for the classic process
        # elapsed_time_process2 = end_time_process2 - end_time_process1
        # cl_time[i_index, j] = elapsed_time_process2

        print('successful ratio is: ',succ_ratio)
        print('solution quality is: ',solution_quality)
        print('minimum number of bin is',minbin)
        # print("Elapsed time for quantum process:", np.mean(elapsed_time_process1), "seconds")
        # print("Elapsed time for classic process:", np.mean(elapsed_time_process2), "seconds")

 50%|█████     | 5/10 [00:02<00:02,  2.13it/s]


successful ratio is:  0.9791666666666666
solution quality is:  0.6351351351351351
minimum number of bin is 5


 50%|█████     | 5/10 [00:02<00:02,  1.83it/s]


successful ratio is:  1.0
solution quality is:  0.676056338028169
minimum number of bin is 5


 50%|█████     | 5/10 [00:02<00:02,  1.72it/s]


...

successful ratio is:  0.6666666666666666
solution quality is:  0.8205128205128205
minimum number of bin is 6


 50%|█████     | 5/10 [00:00<00:00, 35.30it/s]


successful ratio is:  0.6458333333333334
solution quality is:  0.8611111111111112
minimum number of bin is 5


 50%|█████     | 5/10 [00:00<00:00, 18.89it/s]

successful ratio is:  0.7291666666666666
solution quality is:  0.9210526315789473
minimum number of bin is 5


In [17]:
# Write data to a text file
with open('qaoa-k-W2.txt', 'w') as f:
    f.write("dk, batch, sr_set, sq_set, minbin_set, n_solu\n")
    for i_index, i in enumerate(dk_list):
        for j in range(batch):
            f.write(f"{i}, {j}, {sr_set[i_index, j]}, {sq_set[i_index, j]}, {minbin_set[i_index, j]}, {n_solu_set[i_index, j]}\n")

In [9]:
#dk = 1.5
ite = 100
n_layer = 1

dk_list = [4]
#iteration_list = [100, 150, 200]
#n_layer_list = [1, 2, 3]

In [10]:
n_items = 10 #number of items
capacity = 120 #maximum capacity of box

# insert QAOA weight list to compare fairly
W = [28, 30, 31, 36, 46, 32, 49, 29, 33, 21]


#np.random.normal(loc=capacity/2, size=n)
beta = np.min(W)/5
delta_w = min_positive_difference(W)

# generate exact solutions
result_combinations = find_combinations(W, capacity)
target_solu = list(result_combinations)
target = list(range(n_items))

In [11]:
len(target_solu)

205

In [12]:
batch = 5
sr_set = np.zeros((len(dk_list), batch))
sq_set = np.zeros((len(dk_list), batch))
n_solu_set = np.zeros((len(dk_list), batch))
minbin_set = np.zeros((len(dk_list), batch))


for i_index, dk in enumerate(dk_list):

    for j in range(batch):

        solution_set = []
        # qu_time = []
        # start_time_process1 = time.time()

        for k in tqdm(np.arange(1,(capacity/delta_w),dk)):
            alpha = 2*beta*(capacity-k*delta_w)
            h_p = generate_h_two_body(n_items, W, beta) + generate_h_one_body(n_items, W, alpha, beta, capacity)
            h_m = generate_h_mixer(n_items)
            h_cd = commutator(h_m,h_p)#CD pool
            cd = CD_operator(h_cd)
            ci = CD_ansataz(cd)

            circ = Quantum_QAOA_Circuits(h_m, h_p, n_layer, n_items)
            #ground_energy.append(E0_energy(h_p))

            ham = Hamiltonian(h_p)
            sim = Simulator('mqvector', n_items)
            grad_ops = sim.get_expectation_with_grad(ham, circ)

            pr, result = train(ham, circ, ite)
            #cost_fun.append(result)
            res = state_sampling(circ, pr)
            solution_set.append(res.data)

        # Record the current time after the first process
        # end_time_process1 = time.time()

        # Calculate the elapsed time for the quantum process
        # elapsed_time_process1 = end_time_process1 - start_time_process1
        # qu_time[i_index, j] = elapsed_time_process1

        elements_fea, succ_ratio, solution_quality= data_analyze(n_items, solution_set, W[::-1], capacity, target_solu)
        positions = [[n_items-1-i for i, char in enumerate(element) if char == '1'] for element in elements_fea]
        minbin, n_solu = find_min_number(positions,target)
        
        minbin_set[i_index, j] = minbin
        n_solu_set[i_index, j] = n_solu
        sr_set[i_index, j] = succ_ratio
        sq_set[i_index, j] = solution_quality

        # Record the current time before the second process
        # end_time_process2 = time.time()

        # Calculate the elapsed time for the classic process
        # elapsed_time_process2 = end_time_process2 - end_time_process1
        # cl_time[i_index, j] = elapsed_time_process2

        print('successful ratio is: ',succ_ratio)
        print('solution quality is: ',solution_quality)
        print('minimum number of bin is',minbin)
        # print("Elapsed time for quantum process:", np.mean(elapsed_time_process1), "seconds")
        # print("Elapsed time for classic process:", np.mean(elapsed_time_process2), "seconds")

 30%|███       | 3/10 [00:00<00:00, 38.16it/s]


successful ratio is:  0.3463414634146341
solution quality is:  0.8658536585365854
minimum number of bin is 3


 40%|████      | 4/10 [00:01<00:01,  3.40it/s]


successful ratio is:  0.35609756097560974
solution quality is:  0.8690476190476191
minimum number of bin is 4


 30%|███       | 3/10 [00:00<00:00, 28.59it/s]


successful ratio is:  0.37073170731707317
solution quality is:  0.8837209302325582
minimum number of bin is 3


 30%|███       | 3/10 [00:00<00:00, 22.81it/s]


successful ratio is:  0.4292682926829268
solution quality is:  0.88
minimum number of bin is 3


 30%|███       | 3/10 [00:00<00:00, 55.85it/s]

successful ratio is:  0.4292682926829268
solution quality is:  0.88
minimum number of bin is 3


In [21]:
# Write data to a text file
with open('qaoa-k-W3.txt', 'w') as f:
    f.write("dk, batch, sr_set, sq_set, minbin_set, n_solu\n")
    for i_index, i in enumerate(dk_list):
        for j in range(batch):
            f.write(f"{i}, {j}, {sr_set[i_index, j]}, {sq_set[i_index, j]}, {minbin_set[i_index, j]}, {n_solu_set[i_index, j]}\n")

In [22]:
#dk = 1.5
ite = 100
n_layer = 1

dk_list = [0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4]
#iteration_list = [100, 150, 200]
#n_layer_list = [1, 2, 3]

In [38]:
n_items = 10 #number of items
capacity = 120 #maximum capacity of box

# insert QAOA weight list to compare fairly
W = [41, 31, 30, 26, 21, 24, 31, 30, 42, 29]

#np.random.normal(loc=capacity/2, size=n)
beta = np.min(W)/5
delta_w = min_positive_difference(W)

# generate exact solutions
result_combinations = find_combinations(W, capacity)
target_solu = list(result_combinations)
target = list(range(n_items))

In [39]:
len(target_solu)

265

In [24]:
batch = 5
sr_set = np.zeros((len(dk_list), batch))
sq_set = np.zeros((len(dk_list), batch))
n_solu_set = np.zeros((len(dk_list), batch))
minbin_set = np.zeros((len(dk_list), batch))


for i_index, dk in enumerate(dk_list):

    for j in range(batch):

        solution_set = []
        # qu_time = []
        # start_time_process1 = time.time()

        for k in tqdm(np.arange(1,(capacity/delta_w),dk)):
            alpha = 2*beta*(capacity-k*delta_w)
            h_p = generate_h_two_body(n_items, W, beta) + generate_h_one_body(n_items, W, alpha, beta, capacity)
            h_m = generate_h_mixer(n_items)
            h_cd = commutator(h_m,h_p)#CD pool
            cd = CD_operator(h_cd)
            ci = CD_ansataz(cd)

            circ = Quantum_QAOA_Circuits(h_m, h_p, n_layer, n_items)
            #ground_energy.append(E0_energy(h_p))

            ham = Hamiltonian(h_p)
            sim = Simulator('mqvector', n_items)
            grad_ops = sim.get_expectation_with_grad(ham, circ)

            pr, result = train(ham, circ, ite)
            #cost_fun.append(result)
            res = state_sampling(circ, pr)
            solution_set.append(res.data)

        # Record the current time after the first process
        # end_time_process1 = time.time()

        # Calculate the elapsed time for the quantum process
        # elapsed_time_process1 = end_time_process1 - start_time_process1
        # qu_time[i_index, j] = elapsed_time_process1

        elements_fea, succ_ratio, solution_quality= data_analyze(n_items, solution_set, W[::-1], capacity, target_solu)
        positions = [[n_items-1-i for i, char in enumerate(element) if char == '1'] for element in elements_fea]
        minbin, n_solu = find_min_number(positions,target)
        
        minbin_set[i_index, j] = minbin
        n_solu_set[i_index, j] = n_solu
        sr_set[i_index, j] = succ_ratio
        sq_set[i_index, j] = solution_quality

        # Record the current time before the second process
        # end_time_process2 = time.time()

        # Calculate the elapsed time for the classic process
        # elapsed_time_process2 = end_time_process2 - end_time_process1
        # cl_time[i_index, j] = elapsed_time_process2

        print('successful ratio is: ',succ_ratio)
        print('solution quality is: ',solution_quality)
        print('minimum number of bin is',minbin)
        # print("Elapsed time for quantum process:", np.mean(elapsed_time_process1), "seconds")
        # print("Elapsed time for classic process:", np.mean(elapsed_time_process2), "seconds")

 30%|███       | 3/10 [00:01<00:03,  2.32it/s]


successful ratio is:  0.9358490566037736
solution quality is:  0.7630769230769231
minimum number of bin is 3


 30%|███       | 3/10 [00:01<00:02,  2.49it/s]


successful ratio is:  0.9132075471698113
solution quality is:  0.7586206896551724
minimum number of bin is 3


 30%|███       | 3/10 [00:04<00:11,  1.64s/it]


...

successful ratio is:  0.37358490566037733
solution quality is:  0.8608695652173913
minimum number of bin is 3


 30%|███       | 3/10 [00:00<00:00, 37.69it/s]


successful ratio is:  0.37358490566037733
solution quality is:  0.853448275862069
minimum number of bin is 3


 30%|███       | 3/10 [00:00<00:00, 39.83it/s]

successful ratio is:  0.36981132075471695
solution quality is:  0.8828828828828829
minimum number of bin is 3


In [25]:
# Write data to a text file
with open('qaoa-k-W4.txt', 'w') as f:
    f.write("dk, batch, sr_set, sq_set, minbin_set, n_solu\n")
    for i_index, i in enumerate(dk_list):
        for j in range(batch):
            f.write(f"{i}, {j}, {sr_set[i_index, j]}, {sq_set[i_index, j]}, {minbin_set[i_index, j]}, {n_solu_set[i_index, j]}\n")

In [26]:
#dk = 1.5
ite = 100
n_layer = 1

dk_list = [0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4]
#iteration_list = [100, 150, 200]
#n_layer_list = [1, 2, 3]

In [40]:
n_items = 10 #number of items
capacity = 120 #maximum capacity of box

# insert QAOA weight list to compare fairly
W = [42, 55, 52, 80, 55, 80, 53, 46, 79, 66]

#np.random.normal(loc=capacity/2, size=n)
beta = np.min(W)/5
delta_w = min_positive_difference(W)

# generate exact solutions
result_combinations = find_combinations(W, capacity)
target_solu = list(result_combinations)
target = list(range(n_items))

In [41]:
len(target_solu)

29

In [28]:
batch = 5
sr_set = np.zeros((len(dk_list), batch))
sq_set = np.zeros((len(dk_list), batch))
n_solu_set = np.zeros((len(dk_list), batch))
minbin_set = np.zeros((len(dk_list), batch))


for i_index, dk in enumerate(dk_list):

    for j in range(batch):

        solution_set = []
        # qu_time = []
        # start_time_process1 = time.time()

        for k in tqdm(np.arange(1,(capacity/delta_w),dk)):
            alpha = 2*beta*(capacity-k*delta_w)
            h_p = generate_h_two_body(n_items, W, beta) + generate_h_one_body(n_items, W, alpha, beta, capacity)
            h_m = generate_h_mixer(n_items)
            h_cd = commutator(h_m,h_p)#CD pool
            cd = CD_operator(h_cd)
            ci = CD_ansataz(cd)

            circ = Quantum_QAOA_Circuits(h_m, h_p, n_layer, n_items)
            #ground_energy.append(E0_energy(h_p))

            ham = Hamiltonian(h_p)
            sim = Simulator('mqvector', n_items)
            grad_ops = sim.get_expectation_with_grad(ham, circ)

            pr, result = train(ham, circ, ite)
            #cost_fun.append(result)
            res = state_sampling(circ, pr)
            solution_set.append(res.data)

        # Record the current time after the first process
        # end_time_process1 = time.time()

        # Calculate the elapsed time for the quantum process
        # elapsed_time_process1 = end_time_process1 - start_time_process1
        # qu_time[i_index, j] = elapsed_time_process1

        elements_fea, succ_ratio, solution_quality= data_analyze(n_items, solution_set, W[::-1], capacity, target_solu)
        positions = [[n_items-1-i for i, char in enumerate(element) if char == '1'] for element in elements_fea]
        minbin, n_solu = find_min_number(positions,target)
        
        minbin_set[i_index, j] = minbin
        n_solu_set[i_index, j] = n_solu
        sr_set[i_index, j] = succ_ratio
        sq_set[i_index, j] = solution_quality

        # Record the current time before the second process
        # end_time_process2 = time.time()

        # Calculate the elapsed time for the classic process
        # elapsed_time_process2 = end_time_process2 - end_time_process1
        # cl_time[i_index, j] = elapsed_time_process2

        print('successful ratio is: ',succ_ratio)
        print('solution quality is: ',solution_quality)
        print('minimum number of bin is',minbin)
        # print("Elapsed time for quantum process:", np.mean(elapsed_time_process1), "seconds")
        # print("Elapsed time for classic process:", np.mean(elapsed_time_process2), "seconds")

 70%|███████   | 7/10 [00:01<00:00,  4.18it/s]


successful ratio is:  0.9655172413793104
solution quality is:  0.5384615384615384
minimum number of bin is 7


 70%|███████   | 7/10 [00:02<00:00,  3.19it/s]


successful ratio is:  1.0
solution quality is:  0.5918367346938775
minimum number of bin is 7


 70%|███████   | 7/10 [00:02<00:00,  3.21it/s]


...

successful ratio is:  0.7931034482758621
solution quality is:  0.7931034482758621
minimum number of bin is 7


 70%|███████   | 7/10 [00:00<00:00, 38.79it/s]


successful ratio is:  0.7241379310344828
solution quality is:  0.7777777777777778
minimum number of bin is 7


 70%|███████   | 7/10 [00:00<00:00, 26.09it/s]

successful ratio is:  0.7586206896551724
solution quality is:  0.7586206896551724
minimum number of bin is 7


In [29]:
# Write data to a text file
with open('qaoa-k-W5.txt', 'w') as f:
    f.write("dk, batch, sr_set, sq_set, minbin_set, n_solu\n")
    for i_index, i in enumerate(dk_list):
        for j in range(batch):
            f.write(f"{i}, {j}, {sr_set[i_index, j]}, {sq_set[i_index, j]}, {minbin_set[i_index, j]}, {n_solu_set[i_index, j]}\n")

In [ ]:
gs = E0_energy(h_p)

In [ ]:
plt.xlabel('iteration', fontsize=12)
plt.ylabel('Energy', fontsize=12)
plt.title("100 iteration", fontsize=12)
plt.axhline(y=gs, color="black", linestyle="--", label="True energy")
plt.plot(result)
plt.savefig('100-dc-p=1.png')

In [ ]:
yz = QubitOperator(f'Y0 Z1',1)
c = Circuit()
c += decompose_single_term_time_evolution(yz, ParameterResolver(f'b{index}'))

In [ ]:
c.svg().to_file('yz_0314.svg')